**!** *Tip: type `shift+enter` OR click the ` >| Run ` button above to exectue a cell*

### Init

In [5]:
import sys
sys.path.append("./src")

from models import *
from processing import *

### Setup (change initialization cell)

#### OLIM Coil Parameters

In [6]:
coil = Coil(wired_in=Coil.SERIES, current=Coil.DC, material=Coil.COPPER, wire_geometry=Coil.CIRCLE)
tube = Tube(ID=18, OD=21, effective_length=150, unit="mm")

In [7]:
p = Process(coil, tube)

### Exploring Parameters

#### wire diameter and coil height

Explore against one or multiple parameters (ctrl/Command+click, or shift+click)

In [8]:
p.explore_wire_diameter(resolution=0.001, debug=False)

GridBox(children=(GridBox(children=(Select(disabled=True, index=3, layout=Layout(grid_area='resistance', heigh…

After exploring, assign the parameters:

In [9]:
coil.set_wire_d(1.0, unit="mm") 
coil.set_layer_count(14)  # comment if you wish to explore this value
coil.mag_Br = 1.45  #  N45 neodynium magnet

Once we have explored values for wire diameter and current, 
we can explore force against extension of plunger and layer height

### Explore Actuator Force

In [10]:
p.explore_actuator_force()

# Source of equations https://core.ac.uk/download/pdf/9554327.pdf page 14
# https://danielteodesigntechnology.wordpress.com/theoretical/supplement-lorentz-force-actuation/

GridBox(children=(HTML(value='Coil Layers', layout=Layout(grid_area='title', width='99%')), FloatRangeSlider(v…

### Playground

In [30]:
coil.set_wire_d(1.0, unit='mm')

coil.set_layer_count(14)
# OR
#  coil.set_layer_height(14.0, unit='mm')
#  this will automatically calculate layer count

Electrical Properties

In [31]:
I, R = coil.current, coil.resistance; V = I * R
p.out(u"Max Parameters:\n"
       "V=%0.8fV \n"
       "I=%0.2fA \n"
       "R=%0.10f\u2126 \n"
       "P=%0.2fW" % (V, I, R, V*I))

HTML(value='Max Parameters:<br/>V=7.65951705V <br/>I=1.96A <br/>R=3.9009600000Ω <br/>P=15.04W', layout=Layout(…

Physical Properties

In [32]:
wire_d, coil_m, coil_L, coil_h, N_turns = coil.get_wire_d(unit='mm'), \
                                          coil.get_mass(unit='kg'), \
                                          coil.get_length(unit='m'), \
                                          coil.get_height(unit="mm"), \
                                          coil.get_number_of_turns()
p.out("wire diameter=%.2fmm \n"
      "coil mass=%.2fkg \n"
      "coil length=%.1fm \n"
      "coil OD=%.2fmm \n"
      "number of turns=%.0f"% (wire_d, coil_m, coil_L, coil_h*2 + tube.OD*1000, N_turns))

HTML(value='wire diameter=1.00mm <br/>coil mass=1.25kg <br/>coil length=0.1m <br/>coil OD=49.00mm <br/>number …

Force of actuator, represented as the vector (magnitude, direction) in the axis of travel.

In [43]:
samples = 19# how many data points over the stroke of the actuator
for z in [(x/(samples-1))*tube.effective_length for x in range(0, samples, 1)]:
    print("x=%.3fm," % z, end=' F=')
#     Bmag = 1.241  #  N38 Neodydlium magnet
    Bmag = 1.35 #  N42 Neodydlium magnet
    Bsol_z = coil.calculate_magnetic_field(_coil_layers=coil.layer_count, z=z)

    d = tube.ID
    A = math.pi*pow(d/2 + (tube.OD - tube.ID)/2 + 0.001, 2)
    mu = 4.0 * math.pi * pow(10, -7)

    f_z = (Bsol_z[0] + (Bsol_z[1])) * Bmag * A / mu
    print("%.3f" % f_z, end=' ')  # resolved
    print("(%.3f, %.3f)" % (Bsol_z[0] * Bmag * A / mu, Bsol_z[1] * Bmag * A / mu))  # components

x=0.000m, F=7.576 (7.576, 7.576)
x=0.008m, F=7.601 (7.601, 6.081)
x=0.017m, F=7.619 (7.619, 4.849)
x=0.025m, F=7.633 (7.633, 3.817)
x=0.033m, F=7.644 (7.644, 2.940)
x=0.042m, F=7.653 (7.653, 2.187)
x=0.050m, F=7.660 (7.660, 1.532)
x=0.058m, F=7.666 (7.666, 0.958)
x=0.067m, F=7.671 (7.671, 0.451)
x=0.075m, F=7.675 (7.675, 0.000)
x=0.083m, F=7.679 (7.679, -0.404)
x=0.092m, F=7.682 (7.682, -0.768)
x=0.100m, F=7.684 (7.684, -1.098)
x=0.108m, F=7.686 (7.686, -1.398)
x=0.117m, F=7.688 (7.688, -1.671)
x=0.125m, F=7.690 (7.690, -1.922)
x=0.133m, F=7.691 (7.691, -2.154)
x=0.142m, F=7.693 (7.693, -2.367)
x=0.150m, F=7.694 (7.694, -2.565)


In [ ]:
F = coil.number_of_turns * 3.1415 * coil.avg_D * 0.25 * 2
p.out("force=%.2fN \n"
      "number of turns=%.0f \n"
      "coil avg D=%.1fmm \n" % (F, coil.number_of_turns, coil.avg_D*1000))

In [25]:
coil.calculate_magnetic_field??